In [27]:
pip install pulp

In [28]:
import numpy as np
import pandas as pd
import kagglehub
import pulp

# --- PASSO 1 e 2: Carregar, Preparar e Segmentar os Dados ---

In [29]:
# Download data frame
path = kagglehub.dataset_download("data855/heart-disease")


In [30]:
data = pd.read_csv('/kaggle/input/heart-disease/heart.csv')

In [31]:
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [32]:
# Dicionários para legibilidade
sex_map = {0: 'Feminino', 1: 'Masculino'}
cp_map = {0: 'Angina Típica', 1: 'Angina Atípica', 2: 'Dor Não Anginosa', 3: 'Assintomático'}

data['sex_label'] = data['sex'].map(sex_map)
data['cp_label'] = data['cp'].map(cp_map)
# Sanitizar os rótulos para remover espaços, garantindo consistência com o PuLP
data['cp_label'] = data['cp_label'].str.replace(' ', '_')

In [33]:
# Agrupar para calcular o tamanho da população e a probabilidade de doença
segments = data.groupby(['sex_label', 'cp_label'])
segment_analysis = segments['target'].agg(['count', 'sum']).rename(columns={'count': 'populacao_total', 'sum': 'casos_positivos'})
segment_analysis['probabilidade_doenca'] = segment_analysis['casos_positivos'] / segment_analysis['populacao_total']

In [34]:
# Lidar com segmentos que podem não ter casos positivos, resultando em NaN/0
segment_analysis = segment_analysis.fillna(0)

print("--- Análise dos Segmentos de Pacientes ---")
print(segment_analysis)
print("\n" + "="*50 + "\n")

--- Análise dos Segmentos de Pacientes ---
                            populacao_total  casos_positivos  \
sex_label cp_label                                             
Feminino  Angina_Atípica                 18               16   
          Angina_Típica                  39               18   
          Assintomático                   4                4   
          Dor_Não_Anginosa               35               34   
Masculino Angina_Atípica                 32               25   
          Angina_Típica                 104               21   
          Assintomático                  19               12   
          Dor_Não_Anginosa               52               35   

                            probabilidade_doenca  
sex_label cp_label                                
Feminino  Angina_Atípica                0.888889  
          Angina_Típica                 0.461538  
          Assintomático                 1.000000  
          Dor_Não_Anginosa              0.971429  
Masculino 

# --- PASSO 3: Definir Parâmetros da Otimização ---

# Definimos o contexto gerencial: orçamento e custos.
# Estes são valores hipotéticos para a simulação.

In [35]:
# Custo hipotético para examinar um paciente de cada segmento.
# Poderia ser o mesmo para todos ou variar. Vamos assumir um custo uniforme.

custo_por_exame = 150  # Custo de R$150 por paciente examinado
segment_analysis['custo'] = custo_por_exame

In [36]:
# Definir a lista de orçamentos que queremos simular
orcamentos_cenarios = [2000, 5000, 10000, 15000, 50000]

#Criar uma lista vazia para armazenar os dicionários de resultados
resultados_finais = []

In [37]:
# Loop principal para iterar sobre cada cenário de orçamento
for orcamento_atual in orcamentos_cenarios:
    print(f"--- Resolvendo para o Orçamento de: R$ {orcamento_atual:,.2f} ---")

    # PASSO 4: Implementação da Otimização com PuLP (dentro do loop) ---
    model = pulp.LpProblem(f"Maximizar_Deteccao_Orcamento_{orcamento_atual}", pulp.LpMaximize)

    segment_names = list(segment_analysis.index)
    variaveis = pulp.LpVariable.dicts("num_examinar", segment_names, lowBound=0, cat='Integer')

    model += pulp.lpSum([segment_analysis.loc[i, 'probabilidade_doenca'] * variaveis[i] for i in segment_names]), "Total_de_Casos_Detectados"

    # A restrição de orçamento agora usa o valor do cenário atual
    model += pulp.lpSum([segment_analysis.loc[i, 'custo'] * variaveis[i] for i in segment_names]) <= orcamento_atual, "Restricao_Orcamento"

    for i in segment_names:
        model += variaveis[i] <= segment_analysis.loc[i, 'populacao_total'], f"Restricao_Populacao_{i[0]}_{i[1]}"

    # PASSO 5: Resolver o Problema e Armazenar os Resultados (dentro do loop)
    model.solve(pulp.PULP_CBC_CMD(msg=0)) # Adicionado msg=0 para um output mais limpo no loop

    if pulp.LpStatus[model.status] == 'Optimal':
        print("  Recomendação de Alocação de Exames por Segmento:")
        alocacao_detalhada_list = []
        for segment_name, var in variaveis.items():
            if var.varValue > 0:
                detalhe_texto = f"{segment_name[0].replace('_', ' ')} - {segment_name[1].replace('_', ' ')}: {int(var.varValue)}"
                print(f"    - {detalhe_texto}")
                alocacao_detalhada_list.append(detalhe_texto)

        # Juntar os detalhes em uma única string para o CSV
        detalhes_alocacao_str = " | ".join(alocacao_detalhada_list)

        total_examinados = sum(var.varValue for var in variaveis.values())
        total_custo = sum(segment_analysis.loc[s, 'custo'] * variaveis[s].varValue for s in segment_names)
        casos_detectados_esperados = pulp.value(model.objective)

        # Adicionar o dicionário com a nova coluna de detalhes à nossa lista
        resultados_finais.append({
            'orcamento_disponivel': orcamento_atual,
            'custo_total_gasto': total_custo,
            'total_pacientes_examinados': int(total_examinados),
            'casos_detectados_esperados': round(casos_detectados_esperados, 2),
            'detalhes_alocacao': detalhes_alocacao_str # NOVA COLUNA
        })
        print("  Cenário calculado com sucesso.\n")
    else:
        print(f"Não foi encontrada uma solução ótima para o orçamento de R$ {orcamento_atual:,.2f}")

# PASSO 6: Exportar os Resultados para CSV
print("\n" + "="*50)
print("Análise de cenários concluída.")

# Converter a lista de dicionários em um DataFrame
resultados_df = pd.DataFrame(resultados_finais)

# Definir o nome do arquivo de saída
nome_arquivo_csv = 'analise_de_cenarios_orcamento.csv'

# Salvar o DataFrame em um arquivo CSV
# Usar sep=';' e decimal=',' é bom para abrir diretamente no Excel em português
resultados_df.to_csv(nome_arquivo_csv, index=False, sep=';', decimal=',')

print(f"Resultados salvos com sucesso no arquivo: {nome_arquivo_csv}")
print("\n--- Tabela de Cenários Gerada ---")
print(resultados_df)

--- Resolvendo para o Orçamento de: R$ 2,000.00 ---
  Recomendação de Alocação de Exames por Segmento:
    - Feminino - Assintomático: 4
    - Feminino - Dor Não Anginosa: 9
  Cenário calculado com sucesso.

--- Resolvendo para o Orçamento de: R$ 5,000.00 ---
  Recomendação de Alocação de Exames por Segmento:
    - Feminino - Assintomático: 4
    - Feminino - Dor Não Anginosa: 29
  Cenário calculado com sucesso.

--- Resolvendo para o Orçamento de: R$ 10,000.00 ---
  Recomendação de Alocação de Exames por Segmento:
    - Feminino - Angina Atípica: 18
    - Feminino - Assintomático: 4
    - Feminino - Dor Não Anginosa: 35
    - Masculino - Angina Atípica: 9
  Cenário calculado com sucesso.

--- Resolvendo para o Orçamento de: R$ 15,000.00 ---
  Recomendação de Alocação de Exames por Segmento:
    - Feminino - Angina Atípica: 18
    - Feminino - Assintomático: 4
    - Feminino - Dor Não Anginosa: 35
    - Masculino - Angina Atípica: 32
    - Masculino - Dor Não Anginosa: 11
  Cenário cal